In [1]:
from dopamine.discrete_domains import run_experiment
import tensorflow as tf
from obstacle_tower_env import ObstacleTowerEnv
import cv2
run_experiment.load_gin_configs(['dopamine/agents/rainbow/configs/rainbow_otc_light_load.gin'], [])


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
#env = ObstacleTowerEnv('../ObstacleTower/obstacletower', worker_id=2)
def run_episode(env):
    done = False
    first_flag = True
    episode_reward = 0.0
    runner = run_experiment.create_runner('hans_agent')
    while not done:
        if first_flag:
            action = runner._initialize_episode()
            first_flag = False
        else:
            action = runner._agent.begin_episode(obs)
        obs, reward, done, info = env.step(action)
        obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
        episode_reward += reward        
    return episode_reward
run_episode(env)

UnityWorkerInUseException: Couldn't start socket communication because worker number 0 is still in use. You may need to manually close a previously opened environment or use a different worker number.
  In call to configurable 'create_otc_environment' (<function create_otc_environment at 0x7fc4687a4ae8>)
  In call to configurable 'Runner' (<function Runner.__init__ at 0x7fc4690ddb70>)
  In call to configurable 'create_runner' (<function create_runner at 0x7fc4690dd9d8>)

In [ ]:
from obstacle_tower_env import ObstacleTowerEnv
import sys
import argparse

def run_episode(env):
    done = False
    episode_reward = 0.0
    
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        episode_reward += reward
        
    return episode_reward

def run_evaluation(env):
    while not env.done_grading():
        run_episode(env)
        env.reset()

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('environment_filename', default='./ObstacleTower/obstacletower', nargs='?')
    parser.add_argument('--docker_training', action='store_true')
    parser.set_defaults(docker_training=False)
    args = parser.parse_args()

    env = ObstacleTowerEnv(args.environment_filename, docker_training=args.docker_training)
    if env.is_grading():
        episode_reward = run_evaluation(env)
    else:
        while True:
            episode_reward = run_episode(env)
            print("Episode reward: " + str(episode_reward))
            env.reset()

    env.close()

